An end-to-end Agentic Fraud Intelligence System using LangGraph, where a traditional ML fraud model feeds SQL-backed analytical agents, a glossary RAG agent answers domain questions from PDFs, and an anomaly agent monitors behavioral drift and raises alerts.

In [1]:
# fraud-agentic-system/
# │
# ├── data/
# │   └── generate_dummy_data.py
# │
# ├── model/
# │   ├── train_fraud_model.ipynb
# │   └── save_scores_to_sql.py
# │
# ├── db/
# │   └── fraud.db
# │
# ├── agents/
# │   ├── router_agent.py
# │   ├── sql_fraud_agent.py
# │   ├── glossary_rag_agent.py
# │   ├── anomaly_agent.py
# │
# ├── pdf/
# │   └── fraud_glossary.pdf
# │
# ├── langgraph_app.py
# ├── requirements.txt
# └── README.md


In [2]:
#CELL 1: Install All Dependencies
!pip install -q pandas numpy scikit-learn sqlalchemy faiss-cpu
!pip install -q langchain langgraph langchain-community langchain-groq
!pip install -q pypdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 102.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 7.1 MB/s eta 0:00:00


In [3]:
import os
os.environ["GROQ_API_KEY"] = " "


In [4]:
#CELL 3: Generate REALISTIC Dummy Transaction Data (1000 rows)
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

np.random.seed(42)

N = 1000

def random_time():
    return datetime.now() - timedelta(minutes=random.randint(0, 100000))

data = {
    "transaction_id": [f"TXN{i}" for i in range(N)],
    "customer_id": np.random.randint(10000, 20000, N),
    "account_id": np.random.randint(50000, 60000, N),
    "merchant_id": np.random.randint(3000, 4000, N),
    "device_id": np.random.randint(100000, 200000, N),

    "transaction_amount": np.round(np.random.exponential(scale=2000, size=N), 2),
    "transaction_type": np.random.choice(["ATM", "POS", "ECOM"], N),
    "channel": np.random.choice(["CARD", "UPI", "NETBANKING"], N),
    "mcc_code": np.random.choice([5411, 5812, 5999, 4829, 5732], N),

    "transaction_hour": np.random.randint(0, 24, N),
    "transaction_day": np.random.randint(1, 31, N),

    "customer_country": np.random.choice(["IN", "US", "AE"], N),
    "merchant_country": np.random.choice(["IN", "US", "AE"], N),

    "txn_count_1hr": np.random.randint(0, 10, N),
    "txn_count_24hr": np.random.randint(1, 50, N),
    "avg_amount_7d": np.round(np.random.uniform(500, 5000, N), 2),
    "amount_deviation": np.round(np.random.uniform(0, 3, N), 2),

    "is_new_device": np.random.choice([0, 1], N, p=[0.85, 0.15]),
    "is_new_merchant": np.random.choice([0, 1], N, p=[0.9, 0.1]),
    "failed_auth_24h": np.random.randint(0, 5, N),
    "card_present": np.random.choice([0, 1], N),

    "transaction_time": [random_time() for _ in range(N)]
}

df = pd.DataFrame(data)

# Fraud label (1% fraud)
df["fraud_label"] = (np.random.rand(N) < 0.01).astype(int)

df.head()


,transaction_id,customer_id,account_id,merchant_id,device_id,transaction_amount,transaction_type,channel,mcc_code,transaction_hour,...,txn_count_1hr,txn_count_24hr,avg_amount_7d,amount_deviation,is_new_device,is_new_merchant,failed_auth_24h,card_present,transaction_time,fraud_label
0,TXN0,17270,59831,3445,130930,2243.86,POS,NETBANKING,5732,10,...,5,10,4383.63,0.84,0,0,2,1,2025-12-18 03:58:33.493017,0
1,TXN1,10860,50827,3669,181835,294.68,ATM,UPI,4829,17,...,8,41,2666.22,1.67,0,0,0,1,2026-01-17 06:39:33.493039,0
2,TXN2,15390,53531,3632,130090,10589.20,ATM,NETBANKING,5732,17,...,2,33,853.37,2.13,0,0,0,0,2026-01-12 02:30:33.493046,0
3,TXN3,15191,57147,3054,187489,1498.30,ECOM,UPI,5812,20,...,6,13,3022.43,0.40,0,0,3,0,2026-01-18 17:23:33.493049,0
4,TXN4,15734,54451,3911,152142,994.69,ECOM,NETBANKING,4829,15,...,5,35,1994.33,2.44,0,0,0,0,2025-12-03 18:50:33.493053,0


In [5]:
#CELL 4: Train Random Forest Fraud Model (Imbalanced)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

df_model = df.copy()

cat_cols = ["transaction_type", "channel", "customer_country", "merchant_country"]
for c in cat_cols:
    df_model[c] = LabelEncoder().fit_transform(df_model[c])

X = df_model.drop(columns=["fraud_label", "transaction_id", "transaction_time"])
y = df_model["fraud_label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

model = RandomForestClassifier(
    n_estimators=200,
    class_weight={0:1, 1:30},
    random_state=42
)

model.fit(X_train, y_train)

print(classification_report(y_test, model.predict(X_test)))


              precision    recall  f1-score   support

           0       0.99      1.00      1.00       199
           1       0.00      0.00      0.00         1

    accuracy                           0.99       200
   macro avg       0.50      0.50      0.50       200
weighted avg       0.99      0.99      0.99       200



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [6]:
#CELL 5: Score Transactions & Store in SQL (SQLite)
from sqlalchemy import create_engine

df["fraud_score"] = model.predict_proba(X)[:, 1]
df["model_version"] = "RF_v1"

engine = create_engine("sqlite:///fraud.db")

df.to_sql("transactions_scored", engine, if_exists="replace", index=False)


1000

In [8]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=c85ed4ee468334b9f3cb60f2678f951a97617db3d927f31dc63d82b6cfaac5aa
  Stored in directory: /root/.cache/pip/wheels/6e/62/11/dc73d78e40a218ad52e7451f30166e94491be013a7850b5d75
Successfully built fpdf


In [9]:
#CELL 6: Create Fraud Glossary PDF
from fpdf import FPDF

pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=11)

content = """
Transaction Fraud:
Unauthorized financial activity performed on an account.

POS (Point of Sale):
Card transaction done at merchant terminal.

Merchant:
Entity accepting card payments.

MCC Code:
Merchant Category Code used to classify merchants.

Velocity Fraud:
Multiple transactions in short time.

Authorization vs Settlement:
Authorization checks funds, settlement moves money.

Insurance Fraud vs Banking Fraud:
Insurance fraud is claim-based, banking fraud is transaction-based.
"""

for line in content.split("\n"):
    pdf.multi_cell(0, 8, line)

pdf.output("fraud_glossary.pdf")


''

In [15]:
!pip install -U langchain-community langchain-huggingface faiss-cpu pypdf


In [18]:
#CELL 7: Build Vector DB from PDF (Glossary Agent)
#from langchain.document_loaders import PyPDFLoader
#from langchain.embeddings import HuggingFaceEmbeddings
#from langchain.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

loader = PyPDFLoader("fraud_glossary.pdf")
docs = loader.load()

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, embeddings)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph
from typing import TypedDict

llm = ChatGroq(model="llama3-70b-8192")

class AgentState(TypedDict):
    query: str
    response: str


In [20]:
def router(state):
    q = state["query"].lower()
    if "top" in q or "fraud transaction" in q:
        return "sql_agent"
    elif "what is" in q or "meaning" in q:
        return "glossary_agent"
    else:
        return "anomaly_agent"


In [21]:
#CELL 10: SQL Fraud Analytics Agent
import pandas as pd

def sql_agent(state):
    query = """
    SELECT transaction_id, transaction_amount, channel, fraud_score
    FROM transactions_scored
    WHERE fraud_label = 1
    ORDER BY fraud_score DESC
    LIMIT 5
    """
    result = pd.read_sql(query, engine)
    return {"response": result.to_string(index=False)}


In [22]:
#CELL 11: Glossary RAG Agent
def glossary_agent(state):
    docs = vectorstore.similarity_search(state["query"], k=2)
    answer = "\n".join([d.page_content for d in docs])
    return {"response": answer}


In [23]:
#CELL 12: Anomaly Detection Agent (Rule-Based)
def anomaly_agent(state):
    q = """
    SELECT AVG(fraud_label) as fraud_rate FROM transactions_scored
    """
    rate = pd.read_sql(q, engine)["fraud_rate"][0]

    if rate > 0.02:
        return {"response": f"🚨 ALERT: Fraud rate high at {rate:.2%}"}
    else:
        return {"response": f"Fraud rate stable at {rate:.2%}"}


In [24]:
#CELL 13: Build LangGraph
graph = StateGraph(AgentState)

graph.add_node("sql_agent", sql_agent)
graph.add_node("glossary_agent", glossary_agent)
graph.add_node("anomaly_agent", anomaly_agent)

graph.set_conditional_entry_point(
    router,
    {
        "sql_agent": "sql_agent",
        "glossary_agent": "glossary_agent",
        "anomaly_agent": "anomaly_agent",
    },
)

graph.set_finish_point("sql_agent")
graph.set_finish_point("glossary_agent")
graph.set_finish_point("anomaly_agent")

app = graph.compile()


In [25]:
#CELL 14: Run the System (Demo)
app.invoke({"query": "Top 5 fraud transactions"})


{'query': 'Top 5 fraud transactions',
 'response': 'transaction_id  transaction_amount    channel  fraud_score\n         TXN29              742.13        UPI        0.660\n        TXN852             1616.39 NETBANKING        0.635\n        TXN508              659.46       CARD        0.615\n        TXN313             1920.74 NETBANKING        0.550\n        TXN421              858.15       CARD        0.000'}

In [26]:
app.invoke({"query": "What is POS transaction?"})


{'query': 'What is POS transaction?',
 'response': 'Transaction Fraud:\nUnauthorized financial activity performed on an account.\nPOS (Point of Sale):\nCard transaction done at merchant terminal.\nMerchant:\nEntity accepting card payments.\nMCC Code:\nMerchant Category Code used to classify merchants.\nVelocity Fraud:\nMultiple transactions in short time.\nAuthorization vs Settlement:\nAuthorization checks funds, settlement moves money.\nInsurance Fraud vs Banking Fraud:\nInsurance fraud is claim-based, banking fraud is transaction-based.'}

In [27]:
app.invoke({"query": "Is fraud behavior changing?"})


{'query': 'Is fraud behavior changing?',
 'response': 'Fraud rate stable at 0.50%'}

In [28]:
# ┌───────────────────────────┐
# │        User Query         │
# └─────────────┬─────────────┘
#               │
#               ▼
# ┌───────────────────────────┐
# │        Router Agent       │
# │  (Intent Classification) │
# │                           │
# │  - SQL Analytics ?        │
# │  - Glossary ?             │
# │  - Anomaly / Alert ?      │
# └─────────────┬─────────────┘
#         ┌─────┴───────────┬───────────────┐
#         │                 │               │
#         ▼                 ▼               ▼
# ┌────────────────┐ ┌────────────────┐ ┌────────────────────┐
# │ Fraud SQL Agent│ │ Glossary Agent  │ │ Anomaly Detection  │
# │ (Agent 1)      │ │ (Agent 2)       │ │ Agent (Agent 3)    │
# │                │ │                  │ │                    │
# │ - Uses SQL     │ │ - Uses PDF RAG   │ │ - Uses SQL         │
# │ - Reads model  │ │ - Definitions   │ │ - Threshold rules  │
# │   outputs      │ │ - Domain terms  │ │ - Pattern change   │
# │ - Top fraud TX │ │                  │ │ - Drift detection │
# └───────┬────────┘ └────────┬─────────┘ └─────────┬──────────┘
#         │                   │                     │
#         └───────────┬───────┴───────────┬─────────┘
#                     ▼                   ▼
#            ┌──────────────────────────────────┐
#            │     Final Summary Agent           │
#            │                                  │
#            │ - Combines agent output           │
#            │ - Human readable explanation      │
#            │ - NO new facts (faithful)         │
#            └───────────────┬──────────────────┘
#                            ▼
#                   ┌─────────────────┐
#                   │   Final Answer  │
#                   └─────────────────┘


In [29]:
#graph
from typing import TypedDict, Literal
import pandas as pd

class FraudState(TypedDict):
    query: str
    intent: Literal["SQL", "GLOSSARY", "ANOMALY"]
    response: str


In [30]:
#Router Agent
def router_agent(state: FraudState) -> FraudState:
    q = state["query"].lower()

    if any(k in q for k in ["top", "list", "fraud transaction", "amount"]):
        state["intent"] = "SQL"

    elif any(k in q for k in ["what is", "meaning", "define", "pos", "merchant"]):
        state["intent"] = "GLOSSARY"

    else:
        state["intent"] = "ANOMALY"

    return state


In [31]:
#Agent 1: Fraud SQL Analytics Agent
def sql_fraud_agent(state: FraudState) -> FraudState:
    query = """
    SELECT transaction_id,
           transaction_amount,
           channel,
           fraud_score
    FROM transactions_scored
    WHERE fraud_label = 1
    ORDER BY fraud_score DESC
    LIMIT 5
    """
    df = pd.read_sql(query, engine)

    state["response"] = "Top Fraud Transactions:\n\n" + df.to_string(index=False)
    return state


In [32]:
#Agent 2: Glossary RAG Agent (PDF)
def glossary_agent(state: FraudState) -> FraudState:
    docs = vectorstore.similarity_search(state["query"], k=2)
    answer = "\n".join(d.page_content for d in docs)

    state["response"] = answer
    return state


In [33]:
#Agent 3: Anomaly / Alert Agent
def anomaly_agent(state: FraudState) -> FraudState:
    q = "SELECT AVG(fraud_label) AS fraud_rate FROM transactions_scored"
    fraud_rate = pd.read_sql(q, engine)["fraud_rate"][0]

    if fraud_rate > 0.02:
        msg = f"🚨 ALERT: Fraud rate spiked to {fraud_rate:.2%}"
    else:
        msg = f"✅ Fraud rate stable at {fraud_rate:.2%}"

    state["response"] = msg
    return state


In [34]:
from langgraph.graph import StateGraph, END
def route_selector(state: FraudState):
    return state["intent"]


In [40]:
#!pip install pygraphviz



In [35]:
graph = StateGraph(FraudState)

# Add nodes
graph.add_node("router", router_agent)
graph.add_node("sql_agent", sql_fraud_agent)
graph.add_node("glossary_agent", glossary_agent)
graph.add_node("anomaly_agent", anomaly_agent)

# Entry point
graph.set_entry_point("router")

# Conditional routing
graph.add_conditional_edges(
    "router",
    route_selector,
    {
        "SQL": "sql_agent",
        "GLOSSARY": "glossary_agent",
        "ANOMALY": "anomaly_agent"
    }
)

# End points
graph.add_edge("sql_agent", END)
graph.add_edge("glossary_agent", END)
graph.add_edge("anomaly_agent", END)

app = graph.compile()


In [42]:

from IPython.display import Markdown

Markdown(app.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	router(router)
	sql_agent(sql_agent)
	glossary_agent(glossary_agent)
	anomaly_agent(anomaly_agent)
	__end__([<p>__end__</p>]):::last
	__start__ --> router;
	router -. &nbsp;ANOMALY&nbsp; .-> anomaly_agent;
	router -. &nbsp;GLOSSARY&nbsp; .-> glossary_agent;
	router -. &nbsp;SQL&nbsp; .-> sql_agent;
	anomaly_agent --> __end__;
	glossary_agent --> __end__;
	sql_agent --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc


<!-- fraud-agentic-system/
│
├── data/
│   └── generate_dummy_data.py
│
├── model/
│   ├── train_fraud_model.ipynb
│   └── save_scores_to_sql.py
│
├── db/
│   └── fraud.db
│
├── agents/
│   ├── router_agent.py
│   ├── sql_fraud_agent.py
│   ├── glossary_rag_agent.py
│   ├── anomaly_agent.py
│
├── pdf/
│   └── fraud_glossary.pdf
│
├── langgraph_app.py
├── requirements.txt
└── README.md -->
